In [ ]:
# check_dict contains {song_title: reg_num}
# clean up reg_num before putting in check_dict PA 976-753 -> PA0000976753
# - (get rid of - by replacing, )

In [133]:
search_term = "hi(1)"
search_term = search_term.split("(")[0]
print(search_term)

hi


In [136]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
from collections import defaultdict

from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font


def check_master(song_title, index_check=False):
    substrings = ['Master','Mast', 'ssh', 'bmi']
    if not index_check:
        return any([substring.lower() in song_title.lower() for substring in substrings])
    else:
        return max([song_title.lower().find(substring.lower()) for substring in substrings])
    
    
def format_copynum(song_title):
    string1 = song_title
    string1 = string1.split("-")
    string_list = []
    for string in string1:
        string_list.append(string.split(" "))

    flatten_list = sum(string_list, [])

    length = 0
    for i in range(len(flatten_list)):
        length += len(flatten_list[i])

    song_name = ""

    for index in range(len(flatten_list)):
        if index == 0:
            song_name += flatten_list[index]
            for j in range(12-length):
                song_name += "0"
            continue
        song_name += flatten_list[index]
        
    return song_name

    
def create_check_dict(testfile_name): 
    require_cols = [2, 3]

    dataframe2 = pd.read_excel(testfile_name + '.xlsx', engine = 'openpyxl', usecols = require_cols, skiprows = 2, header = None).head(-142)
    dataframe2 = dataframe2.fillna("")
    
    song_titles = []
    check_dict = defaultdict(str)

    for index, row in dataframe2.iterrows():
        copy_num = format_copynum(row[3])
        if copy_num[:2] in ["PA", "PAu", "PAU", "SR"]: 
            #check_dict[(index+1,(row[2]).split("-")[1])] = copy_num
            check_dict[(index+1,row[2])] = copy_num
    return check_dict
    
    
def create_output(copyright_dict, outputfile_name, accuracy_check=False):
    wb =  Workbook()
    ws =  wb.active
    headings = ["#","Song","Registration #"]
    ws.append(headings)
    
    if accuracy_check:
        check_dict = create_check_dict("test_Song_Registration Information")
        match = 0
        #print("check_dict:", check_dict)
    
    num_recorded = 0
    for key in copyright_dict.keys():
        #char = get_column_letter(col)
        # easy mvp but need to store original row number and do something like ws[char + row_iter]
        row_num = key[0]
        print("\nrow_num: ", row_num)
        song_title = key[1]
        # want the original song title (in the case of MASTER version of 20th Century Boy and regular 20th Century Boy, for example)
        copy_num = copyright_dict[key]
        #print(song_title, end="")
        #print(copy_num)
        #print("check_dict[row_num, song_title]: ", check_dict[(row_num,song_title)])
        print("song_title: ", song_title)
        if accuracy_check:
            if check_dict[(row_num,song_title)] == " ":
                print("non PA/SR number entry -- won't be factored into total accuracy count")
                num_recorded -= 1
            elif check_dict[(row_num,song_title)][:2] not in ["PA", "SR"]:
                num_recorded -= 1
            else:
                print("copy_num: ", copy_num)
                print("Manual entry (check_dict[row_num, song_title]) is: ", check_dict[(row_num,song_title)])
                if check_dict[(row_num,song_title)].lower() == copy_num.lower():
                    print("match!")
                    match += 1
        
        ws.append([row_num, song_title, copy_num])
        if copy_num != " ":
            num_recorded += 1
    
    wb.save(outputfile_name + '.xlsx')
    accuracy = match/num_recorded * 100
    print("\nCheck out the output file!")
    print("Accounting for", num_recorded, "songs out of", len(copyright_dict))
    print("Match against manually inputted file is: ", accuracy, "%")
    print(match, " matches out of ", num_recorded, " registration numbers recorded")
    
if __name__ == "__main__":
    
    require_cols = [2, 2]
    dataframe1 = pd.read_excel('/Users/xiaoyanyang/Desktop/MusicCatalog_WebCrawler/input_Song_Titles.xlsx', engine = 'openpyxl', usecols = require_cols, skiprows = 2, header = None).head(-142)
    copyright_dict = defaultdict(str)
    #song_titles1 = []
    song_titles = []
    #breaks_code = ["I'M TOO SEXY", 'HUMBLE AND KIND (TIM MCGRAW MA', 'HOLIDAY (SSH)', "CAN'T YOU SEE (U.S. ONLY AS OF)", "HIGHWAY DON'T CARE (TIM MCGRAW", '25 OR 6 TO 4 (GOARMY REMIX)', 'DANCE HALL DAYS (WANG CHUNG RE', 'SHOTGUN RIDER (TIM MCGRAW MAST', 'SHOTGUN RIDER (TIM MCGRAW MAST', 'BAREFOOT BLUE JEAN NIGHT (BMI']
    breaks_code = ["I'M TOO SEXY", 'HUMBLE AND KIND (TIM MCGRAW MA', "HIGHWAY DON'T CARE (TIM MCGRAW", 'HOLIDAY (SSH)', 'BODY', 'LET MY LOVE OPEN THE DOOR', "WON'T GET FOOLED AGAIN", 'LOOK WHAT YOU MADE ME DO', '25 OR 6 TO 4 (GOARMY REMIX)', 'DANCE HALL DAYS (WANG CHUNG RE', 'SHOTGUN RIDER (TIM MCGRAW MAST']
    
    for index, row in dataframe1.iterrows():
        if row[2].split("-")[1] in breaks_code:
            continue
        else:
            #if "(" in row[2]:
            #    first_half = row[2].split("(")[0]
            #    song_titles1.append((first_half.split("-"))[1])
            #else:
            row_item = row[2].replace("(", "")
            #song_titles1.append((index+1,(row_item).split("-")[1]))
            song_titles.append((index+1,row_item))
    
    PATH = "/Users/xiaoyanyang/Desktop/chromedriver"
    driver = webdriver.Chrome(PATH)
    driver.get("https://cocatalog.loc.gov/cgi-bin/Pwebrecon.cgi?DB=local&PAGE=first")
    #song_titles = ["20th Century Boy", "Burn Out", "Glorious Domination (Master)", "More Girls Like You", "Stars in the night"]
    #song_titles = ["Glorious Domination (Master)"]
    #song_titles = ["CAN'T YOU SEE (U.S. ONLY AS OF)"]
    #song_title = ["HUMBLE AND KIND (TIM MCGRAW MA"]
    
    #if "(" in song_title[0]:
        #first_half = row[2].split("(")[0]
    #song_titles.append((1,song_title[0].split("-")[1][:-1])) <-- [:-1] because last element space
    
    #song_title = ["202775-HUMBLE AND KIND (TIM MCGRAW MA"]
    #song_titles.append((6,song_title[0]))
    
    ######## TEST CASE #########
    
    #song_title = ["19819-20th Century Boy - Master"]
    #song_titles.append((12,song_title[0]))
    #if check_master(song_title[0]):
    #    SR_master = True
    #print(song_titles)
    #print(SR_master)
    
    ############################
    
    registered_count = 0
    for i in range(len(song_titles)):
        SR_master = False
        song_index = song_titles[i][0]
        song_title = song_titles[i][1]
        if check_master(song_title):
            SR_master = True
        # search_term can be song title + catalog (try different search terms)
        
        search_term = song_title.split("-")[1]
        #print("\nsearch_term after splitting for -: ", search_term, "\n")
        if check_master(search_term): # gets rid of the phrase master, ssh, etc.
            end_index = check_master(search_term, True)
            search_term = search_term[:end_index-1]
        if search_term[-1] == " ":
            search_term = search_term[:-1]
        
        search_term = search_term.split("(")[0]
            
        #print("\nsearch_term: ", search_term, "\n")
        
        search = driver.find_element_by_name("Search_Arg")
        search.clear()
        search.send_keys(search_term + "\n")
            
        driver.implicitly_wait(0.5)
        
        possibilities_list = []
        
        try:
            table = driver.find_elements_by_xpath("//form/table[2]/tbody/tr")
        except:
            table = driver.find_elements_by_xpath("//form/table[2]")
            
        #try:
        num_cols = len(table[1].text.split(":"))
        #except:
            #search = driver.find_element_by_name("Search_Arg")
            #search.clear()
            #search_term = search_term.split("(")[0]
            #search.send_keys(search_term + "\n")
            #print("\nnew search_term: ", search_term, "\n")
            
        #print("number of cols is", num_cols)
        if num_cols == 2: # details page of single search result case
            row_item = table[2].text
            #print("row item is", row_item)
            try: 
                copy_num = row_item.split(":")[-1].split("/")[-2]
            except:
                # /html/body/form[1]/table[2]/tbody/tr[3]/th
                # need to figure out how to access the column over (while keeping in mind that registration number can show up in different rows)
                row_index = driver.find_elements_by_xpath("//*[contains(text(), 'Registration Number')]").index
                #print(row_index)
            possibilities_list.append(copy_num) 
            
        else: # more than one search result case
            if not i:
                sel = Select(driver.find_element_by_xpath("//select[@name='CNT']"))
                sel.select_by_value("100")
                driver.implicitly_wait(0.5)
                submit_button = driver.find_element_by_xpath("//form/center[2]/form/table/tbody/tr[2]/td/div/div/table/tbody/tr/td[2]/div/input[2]")
                submit = ActionChains(driver)
                submit.click(submit_button)
                submit.perform()
            
            try:
                table = driver.find_elements_by_xpath("//form/table[2]/tbody/tr")
            except:
                table = driver.find_elements_by_xpath("//form/table[2]")
            num_rows = len(table)
            #print("\nnum_rows: ", num_rows, "\n")
            for i in range(1,num_rows):
                row_item = table[i].text 
                copy_num = row_item.split(".")[-1].split(" ")[-2]
                #print("\n\n\n\ncopy_num: " + copy_num + "\n\n\n\n")
                song_info = row_item
                if (search_term.lower() in song_info.lower()):
                    date = row_item.split(".")[-1].split(" ")[-1]
                    if SR_master:
                        #print(SR_master)
                        if copy_num[:2] == "SR":
                            #print("\n\n\n\n------------SR case--------------\n\n\n\n")
                            # possibilities_list.append((copy_num, date)) 
                            # instead of appending date, in better MVP would click into details page 
                            #     and see whether catalog name is listed in authorship on application and append type of work 
                            possibilities_list.append(copy_num)

                    else:
                        if copy_num[:2] == "PA" and copy_num[:3] != "PAu":
                            possibilities_list.append(copy_num) 
        
        #{(song name, row num):copyright #} dictionary
        # in the excel file row num offset by +2
        if not possibilities_list:
            copyright_dict[(song_index, song_title)] = " "
        else:
            registered_count += 1
            copyright_dict[(song_index, song_title)] = possibilities_list[0]
            #print("registered song")
        
        print("Number of songs registered: ", registered_count)
        print(song_title)
        #print("\npossibilities_list: ", possibilities_list, "\n")
        
        try:
            search_toggle = driver.find_element_by_xpath("//center[2]/font/a[2]/img")
        except:
            search_toggle = driver.find_element_by_xpath("//center[2]/font/img")
        finally:
            back_tosearch = ActionChains(driver)
            back_tosearch.click(search_toggle)
            back_tosearch.perform()
            
    driver.close()
    #print(copyright_dict) 
    #print("copyright_dict: ", copyright_dict)
    
    create_output(copyright_dict, "Registration Numbers", True)

# next steps: 
# yup --> - paste to excel file (just take first element of list)
# - use more complete information of excel sheet (catalog name, artist name, date)
# - figure out how to seek registration number dynamically within details page of song
# turn into object-oriented so that things like copyright_dict are global
# build a web interface, start the process after uploading input file and pressing button 
# fix 20th century boy Master version (don't split at the beginning right away on "(" if detect Master)

Number of songs registered:  1
13463-20th Century Boy
Number of songs registered:  2
54307-WHO ARE YOU
Number of songs registered:  3
56418-BABA O'RILEY
Number of songs registered:  4
191848-MEANT TO BE
Number of songs registered:  5
191897-SOUTHBOUND
Number of songs registered:  6
202993-BEER NEVER BROKE MY HEART
Number of songs registered:  7
24776-CAN'T YOU SEE U.S. ONLY AS OF)
Number of songs registered:  8
19819-20th Century Boy - Master
Number of songs registered:  9
11872-Heaven Is A Place On Earth
Number of songs registered:  10
56421-BEHIND BLUE EYES
Number of songs registered:  11
56531-PINBALL WIZARD
Number of songs registered:  12
13262-Children Of The Revolution
Number of songs registered:  13
191840-LOVE WINS
Number of songs registered:  14
1943-GONNA MAKE YOU SWEAT
Number of songs registered:  15
100056-TEACH YOUR CHILDREN
Number of songs registered:  16
12008-You Get What You Give
Number of songs registered:  17
191644-DRINKING ALONE
Number of songs registered:  18
1826

In [30]:
#PA 976-753 -> PA0000976753
string1 = "PA 976-753"
#string1 = "PA0000976753"
#string1 = "PA-877-899"

#string1 = string1.replace("-","")
string1 = string1.split("-")
#[[],[]]
string_list = []
for string in string1:
    string_list.append(string.split(" "))

flatten_list = sum(string_list, [])

length = 0
for i in range(len(flatten_list)):
    length += len(flatten_list[i])

song_name = ""

for index in range(len(flatten_list)):
    if index == 0:
        song_name += flatten_list[index]
        for j in range(12-length):
            song_name += "0"
        continue
    song_name += flatten_list[index]

print(song_name)

PA0000976753


In [126]:
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

def create_output():
    copyright_dict = {(1, '20th Century Boy'): 'V3613D402', (2, 'WHO ARE YOU'): 'SRu000116460', (3, "BABA O'RILEY"): 'SR0000776140', (4, 'MEANT TO BE'): 'SRu000868462', (5, 'HUMBLE AND KIND '): 'SR0000814718', (6, 'LET MY LOVE OPEN THE DOOR'): '[ 2 ] Let my love open the door -- Empty glass ', (7, 'SOUTHBOUND'): 'PA0001149892', (8, 'BEER NEVER BROKE MY HEART'): 'SR0000863063', (9, 'HOLIDAY '): 'RE0000269852', (10, '20th Century Boy '): 'V3613D402', (11, "WON'T GET FOOLED AGAIN"): "[ 2 ] Won't get fooled again. Beautiful loser, dirty white boy ", (12, 'Heaven Is A Place On Earth'): 'TX0005948879', (13, 'BEHIND BLUE EYES'): 'SR0000802916', (14, 'PINBALL WIZARD'): 'PAu003773594', (15, 'Children Of The Revolution'): 'PA0001059197', (16, "HIGHWAY DON'T CARE "): 'V9942D059', (17, 'LOVE WINS'): 'V3603D994', (18, 'GONNA MAKE YOU SWEAT'): 'V3615D973', (19, 'TEACH YOUR CHILDREN'): 'V3615D973', (20, 'You Get What You Give'): 'V3615D973', (21, 'DRINKING ALONE'): 'VA0002145857', (22, 'BODY LIKE A BACK ROAD'): 'SR0000806425', (23, '25 OR 6 TO 4 '): 'V3510D162', (24, 'MY GENERATION'): 'PAu003672704', (25, 'THIS IS IT'): 'SR0000013514', (26, 'DANCE HALL DAYS '): 'PA0000259814', (27, 'THE SEEKER'): 'TX0004936830', (28, 'THE WAY I AM'): 'SR0000639610', (29, 'JOIN TOGETHER'): 'PAu002161082', (30, 'CHA CHA SLIDE'): 'PA0001066911', (31, 'SHOTGUN RIDER '): 'TXu001871010', (32, 'DANCE HALL DAYS'): 'PA0000259814', (33, 'BODY'): 'SRu001075749', (34, 'TOMMY '): 'VAu000525983', (35, "I'M A BOY"): 'SR0000830141', (36, 'EVERYBODY HAVE FUN TONIGHT'): 'PA0000316303', (37, 'CHRISTMAS WRAPPING'): 'VA0000884159', (38, 'HERE AND NOW'): 'SRu001415475', (39, 'FAMILY AFFAIR '): 'SRu001415475', (40, 'BAREFOOT BLUE JEAN NIGHT '): 'V3618D443', (41, 'SUNRISE SUNBURN SUNSET'): 'VA0002130667', (42, 'GOOD VIBES'): 'V3441D410', (43, "RIDIN' ROADS"): 'SR0000845217', (44, "I HOPE YOU'RE HAPPY NOW"): 'V3499D906', (45, "LET'S TWIST AGAIN"): 'TX0002685123', (46, 'I Know You Want Me'): 'TX0002685123', (47, 'EMINENCE FRONT'): 'PAu002848829'}
    wb =  Workbook()
    ws =  wb.active
    headings = ["#","Song","Registration #"]
    ws.append(headings)
    
    for ((row_num, song_title), copy_num) in copyright_dict.items():
        #char = get_column_letter(col)
        # easy mvp but need to store original row number and do something like ws[char + row_iter]
        ws.append([row_num, song_title, copy_num])
    
    wb.save('NewGrades.xlsx')

if __name__ == "__main__":
    create_output()

In [130]:
lst = []
print(not lst)

lst1 = [2]
print(not lst1)

True
False


In [51]:
!pip3 install openpyxl

In [71]:
import pandas as pd
from collections import defaultdict

def format_copynum(song_title):
    string1 = song_title
    string1 = string1.split("-")
    string_list = []
    for string in string1:
        string_list.append(string.split(" "))

    flatten_list = sum(string_list, [])

    length = 0
    for i in range(len(flatten_list)):
        length += len(flatten_list[i])

    song_name = ""

    for index in range(len(flatten_list)):
        if index == 0:
            song_name += flatten_list[index]
            for j in range(12-length):
                song_name += "0"
            continue
        song_name += flatten_list[index]
        
    return song_name

def check_accuracy():
    require_cols = [2, 3]

    dataframe2 = pd.read_excel('/Users/xiaoyanyang/Desktop/MusicCatalog_WebCrawler/test_Song_Registration Information.xlsx', engine = 'openpyxl', usecols = require_cols, skiprows = 2, header = None).head(-142)
    dataframe2 = dataframe2.fillna(" ")
    return dataframe2
    
song_titles = []
check_dict = defaultdict(str)
dataframe1 = check_accuracy()
for index, row in dataframe1.iterrows():
    #print(row[[2,3]])
    #song_titles.append((index+1,(row[2]).split("-")[1], row[3]))
    copy_num = format_copynum(row[3])
    if copy_num[:2] in ["PA", "PAu", "PAU", "SR"]: 
        check_dict[(index+1,(row[2]).split("-")[1])] = copy_num
#print(song_titles)
print(len(check_dict))
print(check_dict)




37
defaultdict(<class 'str'>, {(1, '20th Century Boy'): 'PA0000976753', (2, 'WHO ARE YOU'): 'PAu000051060', (4, 'MEANT TO BE'): 'PA0002105522', (5, "I'M TOO SEXY"): 'PA0000550414', (6, 'HUMBLE AND KIND (TIM MCGRAW MA'): 'PA0002130282', (7, 'LET MY LOVE OPEN THE DOOR'): 'PAu000195123', (8, 'SOUTHBOUND'): 'PAu003940558', (9, 'BEER NEVER BROKE MY HEART'): 'PA0002192567', (12, '20th Century Boy '): 'SR0000066544', (14, 'Heaven Is A Place On Earth'): 'PA0000877899', (17, 'Children Of The Revolution'): 'PA0001015278', (18, "HIGHWAY DON'T CARE (TIM MCGRAW"): 'PA0001865121', (19, 'LOVE WINS'): 'PA0002152848', (20, 'LOOK WHAT YOU MADE ME DO'): 'PA0002087339', (21, 'GONNA MAKE YOU SWEAT'): 'PA0000520228', (23, 'You Get What You Give'): 'PA0000925924', (24, 'DRINKING ALONE'): 'PAu003940536', (25, 'BODY LIKE A BACK ROAD'): 'PA0002067268', (26, '25 OR 6 TO 4 (GOARMY REMIX)'): 'PA0000316591', (28, 'THIS IS IT'): 'PA0002151850', (29, 'DANCE HALL DAYS (WANG CHUNG RE'): 'PA0000215280', (31, 'THE WAY I 

In [8]:
import pandas as pd
  
require_cols = [2, 2]
#new_columns = ["Row Index", "Song Title"]
  
dataframe1 = pd.read_excel('/Users/xiaoyanyang/Desktop/MusicCatalog_WebCrawler/input_Song_Titles.xlsx', engine = 'openpyxl', usecols = require_cols, skiprows = 2, header = None).head(-142)

print(dataframe1)
#print(list(dataframe1.columns))

                                        2
0                  13463-20th Century Boy
1                       54307-WHO ARE YOU
2                      56418-BABA O'RILEY
3                      191848-MEANT TO BE
4                      12279-I'M TOO SEXY
5   202775-HUMBLE AND KIND (TIM MCGRAW MA
6         54222-LET MY LOVE OPEN THE DOOR
7                       191897-SOUTHBOUND
8        202993-BEER NEVER BROKE MY HEART
9                       987-HOLIDAY (SSH)
10  24776-CAN'T YOU SEE (U.S. ONLY AS OF)
11        19819-20th Century Boy - Master
12           56402-WON'T GET FOOLED AGAIN
13       11872-Heaven Is A Place On Earth
14                 56421-BEHIND BLUE EYES
15                   56531-PINBALL WIZARD
16       13262-Children Of The Revolution
17  202740-HIGHWAY DON'T CARE (TIM MCGRAW
18                       191840-LOVE WINS
19        159744-LOOK WHAT YOU MADE ME DO
20              1943-GONNA MAKE YOU SWEAT
21             100056-TEACH YOUR CHILDREN
22            12008-You Get What Y

In [12]:
song_titles = []
for index, row in dataframe1.iterrows():
    song_titles.append((index+1,(row[2]).split("-")[1]))
print(song_titles)

[(1, '20th Century Boy'), (2, 'WHO ARE YOU'), (3, "BABA O'RILEY"), (4, 'MEANT TO BE'), (5, "I'M TOO SEXY"), (6, 'HUMBLE AND KIND (TIM MCGRAW MA'), (7, 'LET MY LOVE OPEN THE DOOR'), (8, 'SOUTHBOUND'), (9, 'BEER NEVER BROKE MY HEART'), (10, 'HOLIDAY (SSH)'), (11, "CAN'T YOU SEE (U.S. ONLY AS OF)"), (12, '20th Century Boy '), (13, "WON'T GET FOOLED AGAIN"), (14, 'Heaven Is A Place On Earth'), (15, 'BEHIND BLUE EYES'), (16, 'PINBALL WIZARD'), (17, 'Children Of The Revolution'), (18, "HIGHWAY DON'T CARE (TIM MCGRAW"), (19, 'LOVE WINS'), (20, 'LOOK WHAT YOU MADE ME DO'), (21, 'GONNA MAKE YOU SWEAT'), (22, 'TEACH YOUR CHILDREN'), (23, 'You Get What You Give'), (24, 'DRINKING ALONE'), (25, 'BODY LIKE A BACK ROAD'), (26, '25 OR 6 TO 4 (GOARMY REMIX)'), (27, 'MY GENERATION'), (28, 'THIS IS IT'), (29, 'DANCE HALL DAYS (WANG CHUNG RE'), (30, 'THE SEEKER'), (31, 'THE WAY I AM'), (32, 'JOIN TOGETHER'), (33, 'CHA CHA SLIDE'), (34, 'SHOTGUN RIDER (TIM MCGRAW MAST'), (35, 'DANCE HALL DAYS'), (36, 'BODY

In [96]:
import openpyxl.utils.dataframe 
from openpyxl import Workbook, load_workbook
from itertools import islice
import pandas as pd

wb = Workbook()
wb =  load_workbook('/Users/xiaoyanyang/Desktop/MusicCatalog_WebCrawler/input_Song_Titles.xlsx')
ws = wb.active
data = ws.values

In [76]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time

In [95]:
PATH = "/Users/xiaoyanyang/Desktop/chromedriver"
driver = webdriver.Chrome(PATH)
driver.get("https://cocatalog.loc.gov/cgi-bin/Pwebrecon.cgi?DB=local&PAGE=first")

In [96]:
search = driver.find_element_by_name("Search_Arg")
search.clear()
search.send_keys("20th Century Boy\n")

In [97]:
driver.implicitly_wait(0.5)
time.sleep(0.5)
#time.sleep(5) 
#try:
    #table = WebDriverWait(driver, 10).until(
        #EC.presence_of_element_located((By.LINK_TEXT, "Beginner Python Tutorials"))
#    )
sel = Select(driver.find_element_by_xpath("//select[@name='CNT']"))
sel.select_by_value("100")
time.sleep(0.8)
submit_button = driver.find_element_by_xpath("//form/center[2]/form/table/tbody/tr[2]/td/div/div/table/tbody/tr/td[2]/div/input[2]").click() 
submit = ActionChains(driver)
submit.click(submit_button)
submit.perform()

In [98]:
table = driver.find_elements_by_xpath("//form/table[2]/tbody/tr")
num_rows = len(table)
print("num of rows is: ", num_rows)
for i in range(num_rows):
    row_item = table[i].text
    print(row_item)

num of rows is:  32
# Title < Full Title Copyright Number Date
[ 1 ] 20th Century Boy. 20th Century Boy. V3615D975 2012
[ 2 ] 20th Century Boy. 20th Century Boy. V3615D975 2012
[ 3 ] 20TH CENTURY BOY. 20TH CENTURY BOY. V3598D036 2011
[ 4 ] 20TH CENTURY BOY. 20TH CENTURY BOY. V3598D037 2011
[ 5 ] 20TH CENTURY BOY. 20TH CENTURY BOY. V3598D044 2011
[ 6 ] 20th Century Boy. 20th Century Boy. PAu003381057 2009
[ 7 ] 20th century boy. 20th century boy. V3582D425 2009
[ 8 ] 20th century boy. 20th century boy. V3576D005 2009
[ 9 ] 20th century boy. 20th century boy. V3576D006 2009
[ 10 ] 20TH CENTURY BOY. 20TH CENTURY BOY. V3602D215 2009
[ 11 ] 20th century boy. 20th century boy. V3562D695 2008
[ 12 ] 20th century boy. 20th century boy. V3562D696 2008
[ 13 ] 20th century boy. 20th century boy. PA0000976753 1999
[ 14 ] 20th century boy 20th century boy ; Dirt / Lorette Velvette. SR0000233398 1995
[ 15 ] 20th century boy -- Ain't no jive, live! / Bang Tango. SR0000143711 1992
[ 16 ] 20th century 

In [128]:
def check_master(song_title):
    substrings = ['Master','Mast']
    #song_title = '20th Century Boy - MASTER'

    output = any([substring.lower() in song_title.lower() for substring in substrings])
    #print(output)
    return output

In [130]:
num_rows = len(table)
song_title = "20th Century Boy"
print("num of rows is: ", num_rows)
possibilities_list = []
for i in range(1,num_rows):
    row_item = table[i].text
    copy_num = row_item.split(".")[-1].split(" ")[-2]
    song_info = row_item
    #print(song_info)
    if (song_title.lower() in song_info.lower()):
        date = row_item.split(".")[-1].split(" ")[-1]
        if check_master(song_title) and copy_num[:2] == "SR":
            possibilities_list.append((copy_num, date)) 
            
        elif copy_num[:2] == "PA": 
            possibilities_list.append((copy_num, date)) 
            #print("True")
            #print(copy_num)
            #print(row_item.split("."))
    #print(row_item)
print(possibilities_list)
    

num of rows is:  32
[('PAu003381057', '2009'), ('PA0000976753', '1999')]


In [13]:
substrings = ['a', 'b', 'c']
a_string = 'cde'

print(substrings)
print(a_string)

output = any([substring in a_string for substring in substrings])
print(output)

['a', 'b', 'c']
cde
True


In [24]:
substrings = ["Master", "Mast"]
a_string = "Glorious Domination (Master)"

output = [a_string.lower().find(substring.lower()) for substring in substrings]
print(output)
print(a_string[:max(output)-1])

[21, 21]
Glorious Domination 


In [92]:
search_toggle = driver.find_element_by_xpath("//center[2]/font/a[2]/img")
back_tosearch = ActionChains(driver)
back_tosearch.click(search_toggle)
back_tosearch.perform()

In [50]:
xpath1 = "//form/table[2]/tbody/tr["+str(2)+"]/td[3]"
full_title = driver.find_element_by_xpath(xpath1).text
print(row_item)

[ 25 ] 20th Century boy. PA 976-753. 20th Century boy. PA 976-753. V3570D349 2008


In [43]:
row_item = driver.find_element_by_xpath("//form/table[2]/tbody/tr[11]/td[4]").text
print(row_item)

V3602D215


In [ ]:
num_rows = len(table)
print("num of rows is: ", num_rows)
for i in range(num_rows):
    xpath1 = "//form/table[2]/tbody/tr["+str(i)+"]/td[3]"
    full_title = driver.find_element_by_xpath(xpath1).text
    xpath2 = "//form/table[2]/tbody/tr["+str(i)+"]/td[4]"
    copy_num = driver.find_element_by_xpath(xpath2).text
    print(full_title, copy_num)
#print("num of columns is: ", len(table[0].split(".")))

### TODO's

MVP: Where only the song name & catalog name is provided, not the date & where we only look at the first 100 search results & we take only the first PA or SR copy # that shows up (no other types of copyright numbers and no considerations for the details page to determine which copyright num is best if multiple possibilities)
---> test against manually inputted data
MVP child: multiple possibilities user prompt
MVP grand-child: google search date of the song & cross-compare for higher accuracy
MVP grand-grand-child: Streamlit interface for webpage: upload file button for excel file and download new Excel file  

1. make ActionChain of search action
2. create dummy CSV of song names & ids
3. iterate through song names & ids and mark down the row number of where we wanna paste the copyright # to
- driver to click on the search button (back to search screen) in between searches 
4. for each song name, do the whole ActionChain (wrap search into ActionChain object), jot down the copyright # in the {(song name, row num):copyright #} dictionary
    - count = 0
    - while count < 3 and !len(possibilities)
        - if condition: if the text saying "displaying # through # of # entries) has the middle # less than the last #, click on the NEXT arrow button 
        - count += 1
    - for cases where there is MASTER or MAST in the title, check for SR only
        - case MULTIPLE
    - else, check for PA only
        - case MULTIPLE
    - for cases where there are no PA or SR, look <-- eh, 
    - case MULTIPLE: for cases where there are multiple PA's or SR's, don't record in dictionary right away
        - come up with list of possibilities by clicking into the page details of a particular song entry if the copyright num is PA for non-MASTER case or SR if MASTER case
            - save down the Type of Work info
        - while user_input != "Y" and count < len(possibilities):
            - Info for this song:
            - Song Name / Catalog / Type of Work:
            - prompt user: "accept " + str(copy_num) + "copyright number, Y/N: "            
        

5. Output: 
    - in python: print out how many songs out of the song titles passed in were registered with PA or SR copyright num (# out of # song titles registered)
    - MVP: 
        - input: just a single column with the id - song title entries
        - output: fill in the adjacent column to the song title column 
    - MVP-child: in excel: fill in the appropriate cells based on the copyright nums recorded

misc. 
- may take a while so do multiple passes
    - 1st pass: only look at first page of search results
- create a summary table of the song titles with master in the name and the songs without